In [1]:
import pandas as pd
from docx import Document
import re
import pandas as pd
import torch
from transformers import BigBirdTokenizer,BigBirdModel,BigBirdForSequenceClassification
from torch import Tensor
import numpy as np
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from typing import list

In [2]:
# Data_csv = pd.read_csv("/home/ubuntu/working_directory/Bert_experimentation/new_dataframe_50000_100000.csv")
Data_csv = pd.read_csv("/home/ubuntu/ritesh_manchikanti/final_data.csv")
# Data_csv.drop('Unnamed: 0', axis=1, inplace=True)
Data_csv

,filename,text,level
0,15031-4983-FullBook.docx,"Learner Choice, Learning Voice\n\n\nLearner Vo...",Level 1
1,15031-4984-FullBook.docx,\n\nExistentialism: A Philosophical Inquiry\n\...,Level 1
2,15031-4985-FullBook.docx,"""The editors of this volume are the top practi...",Level 1
3,15031-4986-FullBook.docx,Black Power Music!\n\nBlack Power Music!: Prot...,Level 1
4,15031-4989-FullBook.docx,Home\n\n\nHome articulates a ‘critical geograp...,Level 1
...,...,...,...
1231,15032-5774-FullBook.docx,The Japanese LGBTQ+ Community in the World\n\n...,Level 3
1232,15032-5789-FullBook.docx,The Acquisition of English Grammar and Phonolo...,Level 3
1233,15032-5829-FullBook.docx,"\n‘New concepts, new words for them, new actio...",Level 3
1234,15032-5843-FullBook.docx,Translating Rumi into the West\n\n\nFocusing o...,Level 3


In [3]:
def split_tokens_into_smaller_chunks(input_id: Tensor,att_mask: Tensor, chunk_size: int, stride: int, minimal_chunk_length: int):
    input_id_chunks = [input_id[i : i + chunk_size] for i in range(0, len(input_id), stride)]
    mask_chunks = [att_mask[i : i + chunk_size] for i in range(0, len(att_mask), stride)]
    if len(input_id_chunks) > 1:
        # ignore chunks with less than minimal_length number of tokens
        input_id_chunks = [x for x in input_id_chunks if len(x) >= minimal_chunk_length]
        mask_chunks = [x for x in mask_chunks if len(x) >= minimal_chunk_length]
    return input_id_chunks, mask_chunks
def add_special_tokens_at_beginning_and_end(input_id_chunks, mask_chunks) -> None:
    """
    Adds special CLS token (token id = 101) at the beginning.
    Adds SEP token (token id = 102) at the end of each chunk.
    Adds corresponding attention masks equal to 1 (attention mask is boolean).
    """
    for i in range(len(input_id_chunks)):
        # adding CLS (token id 101) and SEP (token id 102) tokens
        input_id_chunks[i] = torch.cat([Tensor([101]), input_id_chunks[i], Tensor([102])])
        # adding attention masks  corresponding to special tokens
        mask_chunks[i] = torch.cat([Tensor([1]), mask_chunks[i], Tensor([1])])

def add_padding_tokens(input_id_chunks, mask_chunks) -> None:
    """Adds padding tokens (token id = 0) at the end to make sure that all chunks have exactly 512 tokens."""
    for i in range(len(input_id_chunks)):
        # get required padding length
        pad_len = 512 - input_id_chunks[i].shape[0]
        # check if tensor length satisfies required chunk size
        if pad_len > 0:
            # if padding length is more than 0, we must add padding
            input_id_chunks[i] = torch.cat([input_id_chunks[i], Tensor([0] * pad_len)])
            mask_chunks[i] = torch.cat([mask_chunks[i], Tensor([0] * pad_len)])

def stack_tokens_from_all_chunks(input_id_chunks, mask_chunks):
    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(mask_chunks)
    return input_ids.long(), attention_mask.int()

In [6]:
import pandas as pd
# Read the CSV file containing the filenames
excluded_filenames_df = pd.read_csv('./ddp_2/selected_samples.csv')

# Extract the filenames from the dataframe as a list
excluded_filenames = excluded_filenames_df['filename'].tolist()

# Print the list of excluded filenames
print(excluded_filenames)
print(len(excluded_filenames))

['15032-5430-FullBook.docx', '15032-5915-FullBook.docx', '15032-5390-FullBook.docx', '15032-5267-FullBook.docx', '15032-5641-FullBook.docx', '15032-5891-FullBook.docx', '15032-5595-FullBook.docx', '15032-5356-FullBook.docx', '15032-5252-FullBook.docx', '15032-5847-FullBook.docx', '15032-5261-FullBook.docx', '15032-5735-FullBook.docx', '15032-5248-FullBook.docx', '15031-5195-FullBook.docx', '15032-5282-FullBook.docx', '15032-5960-FullBook.docx', '15032-5548-FullBook.docx', '15032-5408-FullBook.docx', '15032-5446-FullBook.docx', '15031-5481-FullBook.docx', '15032-5705-FullBook.docx', '15032-5364-FullBook.docx', '15031-5241-FullBook.docx', '15031-4999-FullBook.docx', '15032-5273-FullBook.docx', '15032-6003-FullBook.docx', '15031-5548-FullBook.docx', '15032-5830-FullBook.docx', '15032-5748-FullBook.docx', '15032-5344-FullBook.docx', '15032-5251-FullBook.docx', '15032-5350-FullBook.docx', '15032-5619-FullBook.docx', '15032-5392-FullBook.docx', '15032-5764-FullBook.docx', '15032-5316-FullBoo

In [7]:
new_df = Data_csv[~Data_csv['filename'].isin(excluded_filenames)]
new_df
# Print the new dataframe
# print(new_df)

,filename,text,level
0,15031-4983-FullBook.docx,"Learner Choice, Learning Voice\n\n\nLearner Vo...",Level 1
1,15031-4984-FullBook.docx,\n\nExistentialism: A Philosophical Inquiry\n\...,Level 1
2,15031-4985-FullBook.docx,"""The editors of this volume are the top practi...",Level 1
3,15031-4986-FullBook.docx,Black Power Music!\n\nBlack Power Music!: Prot...,Level 1
4,15031-4989-FullBook.docx,Home\n\n\nHome articulates a ‘critical geograp...,Level 1
...,...,...,...
1179,15031-5140-FullBook.docx,Español académico como LE/L2 \n\nEspañol acadé...,Level 3
1187,15031-5238-FullBook.docx,\n\n<<halftitle page i – mandatory element>>\n...,Level 3
1204,15031-5348-FullBook.docx,Access your online resources\n\nCultural Inclu...,Level 3
1208,15031-5407-FullBook.docx,The Uralic Languages\n\n\nThe Uralic Languages...,Level 3


In [10]:

import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn.functional as F
from langchain.text_splitter import TokenTextSplitter
text_splitter_1 = TokenTextSplitter(chunk_size=300, chunk_overlap=128)
text_splitter_2 = TokenTextSplitter(chunk_size=150, chunk_overlap=64)
tokenizer = AutoTokenizer.from_pretrained("./ddp_2/model/")
model = AutoModelForSequenceClassification.from_pretrained("./ddp_2/model/")
label2id = {"Level 1": 0, "Level 2": 1, "Level 3": 2}
id_to_label = {v: k for k, v in label2id.items()}
true_labels = []
predicted_labels = []
print(range(new_df.shape[0]))
for idx in range(new_df.shape[0]):
    print(idx)
    document = new_df["text"].iloc[idx]
    label = label2id[new_df.iloc[idx]["level"]]
    true_labels.append(label)
    chunks = text_splitter_1.split_text(document)
    tokenized_chunks = []
    for chunk in chunks:
        tokens = tokenizer(chunk, truncation=True, padding=True, return_tensors="pt")
        if len(tokens['input_ids'][0]) > 512:
            sub_chunks = text_splitter_2.split_text(chunk)
            print(len(sub_chunks))
            for i in sub_chunks:
                tokens = tokenizer(chunk, truncation=True, padding=True, return_tensors="pt")
                tokenized_chunks.append(tokens)
        else:
            tokenized_chunks.append(tokens)
    chunk_predicted_labels = []
    for tokens in tokenized_chunks:
        outputs = model(**tokens)
        # Get the predicted class index
        predicted_class_index = outputs.logits.argmax().item()
        # Get the predicted class label
        label2id = {"Level 1": 0, "Level 2": 1, "Level 3": 2}
        id_to_label = {v: k for k, v in label2id.items()}
        predicted_class_label = id_to_label[predicted_class_index]
        chunk_predicted_labels.append(predicted_class_label)
    # Find the majority predicted label for all the chunks
    majority_label = max(set(chunk_predicted_labels), key=chunk_predicted_labels.count)
    predicted_labels.append(majority_label)
    print(predicted_class_label)
    print("True label vs predicted label", new_df["filename"].iloc[idx],id_to_label[label],majority_label)
# Count the occurrence of each predicted label
label_counts = pd.Series(predicted_labels).value_counts()
print(label_counts)


range(0, 986)
0


Attention type 'block_sparse' is not possible if sequence_length: 295 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Level 1
True label vs predicted label 15031-4983-FullBook.docx Level 1 Level 3
1
Level 1
True label vs predicted label 15031-4984-FullBook.docx Level 1 Level 1
2
Level 3
True label vs predicted label 15031-4985-FullBook.docx Level 1 Level 1
3
Level 1
True label vs predicted label 15031-4986-FullBook.docx Level 1 Level 1
4
Level 1
True label vs predicted label 15031-4989-FullBook.docx Level 1 Level 1
5
Level 1
True label vs predicted label 15031-4990-FullBook.docx Level 1 Level 2
6
Level 2
True label vs predicted label 15031-4994-FullBook.docx Level 1 Level 1
7
Level 1
True label vs predicted label 15031-4998-FullBook.docx Level 1 Level 1
8
Level 2
True label vs predicted label 15031-5000-FullBook.docx Level 1 Level 1
9
Level 2
True label vs predicted label 15031-5002-FullBook.docx Level 1 Level 1
10
Level 2
True label vs predicted label 15031-5004-FullBook.docx Level 1 Level 1
11
Level 2
True label vs predicted label 15031-5006-FullBook.docx Level 1 Level 1
12
Level 2
True label vs pre

In [ ]:
import docx



# Usage example
docx_file = '/home/ubuntu/ritesh_manchikanti/Bigbird/15031-4983-FullBook.docx'
text = read_docx('/home/ubuntu/ritesh_manchikanti/Bigbird/15031-4983-FullBook.docx')
print(text)

In [17]:

import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn.functional as F
from langchain.text_splitter import TokenTextSplitter
def read_docx(file_path):
    doc = docx.Document(file_path)
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    return '\n'.join(text)
text_splitter_1 = TokenTextSplitter(chunk_size=300, chunk_overlap=128)
text_splitter_2 = TokenTextSplitter(chunk_size=150, chunk_overlap=64)
tokenizer = AutoTokenizer.from_pretrained("./ddp/savedmodel_multi_gpu_ddp/")
model = AutoModelForSequenceClassification.from_pretrained("./ddp/savedmodel_multi_gpu_ddp/")
label2id = {"Level 1": 0, "Level 2": 1, "Level 3": 2}
id_to_label = {v: k for k, v in label2id.items()}
true_labels = []
predicted_labels = []
text = read_docx('/home/ubuntu/ritesh_manchikanti/Bigbird/15031-4983-FullBook.docx')
chunks = text_splitter_1.split_text(document)
tokenized_chunks = []
for chunk in chunks:
    tokens = tokenizer(chunk, truncation=True, padding=True, return_tensors="pt")
    if len(tokens['input_ids'][0]) > 512:
        sub_chunks = text_splitter_2.split_text(chunk)
        print(len(sub_chunks))
        for i in sub_chunks:
            tokens = tokenizer(chunk, truncation=True, padding=True, return_tensors="pt")
            tokenized_chunks.append(tokens)
    else:
        tokenized_chunks.append(tokens)
chunk_predicted_labels = []
for tokens in tokenized_chunks:
    outputs = model(**tokens)
    predicted_class_index = outputs.logits.argmax().item()
    label2id = {"Level 1": 0, "Level 2": 1, "Level 3": 2}
    id_to_label = {v: k for k, v in label2id.items()}
    predicted_class_label = id_to_label[predicted_class_index]
    chunk_predicted_labels.append(predicted_class_label)
majority_label = max(set(chunk_predicted_labels), key=chunk_predicted_labels.count)
# predicted_labels.append(majority_label)
# print(predicted_class_label)
print(majority_label)
# print("True label vs predicted label", new_df["filename"].iloc[idx],id_to_label[label],majority_label)



Attention type 'block_sparse' is not possible if sequence_length: 301 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Level 1


In [19]:
import docx
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from langchain.text_splitter import TokenTextSplitter

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    return '\n'.join(text)

def predict_label(file_path, tokenizer, model):
    text_splitter_1 = TokenTextSplitter(chunk_size=300, chunk_overlap=128)
    text_splitter_2 = TokenTextSplitter(chunk_size=150, chunk_overlap=64)

    label2id = {"Level 1": 0, "Level 2": 1, "Level 3": 2}
    id_to_label = {v: k for k, v in label2id.items()}
    true_labels = []
    predicted_labels = []
    text = read_docx(file_path)

    chunks = text_splitter_1.split_text(text)
    tokenized_chunks = []

    for chunk in chunks:
        tokens = tokenizer(chunk, truncation=True, padding=True, return_tensors="pt")
        if len(tokens['input_ids'][0]) > 512:
            sub_chunks = text_splitter_2.split_text(chunk)
            for sub_chunk in sub_chunks:
                tokens = tokenizer(sub_chunk, truncation=True, padding=True, return_tensors="pt")
                tokenized_chunks.append(tokens)
        else:
            tokenized_chunks.append(tokens)

    chunk_predicted_labels = []

    for tokens in tokenized_chunks:
        outputs = model(**tokens)
        predicted_class_index = outputs.logits.argmax().item()
        predicted_class_label = id_to_label[predicted_class_index]
        chunk_predicted_labels.append(predicted_class_label)

    majority_label = max(set(chunk_predicted_labels), key=chunk_predicted_labels.count)

    return majority_label

# Example usage
tokenizer = AutoTokenizer.from_pretrained("./ddp/savedmodel_multi_gpu_ddp/")
model = AutoModelForSequenceClassification.from_pretrained("./ddp/savedmodel_multi_gpu_ddp/")
file_path = '/home/ubuntu/ritesh_manchikanti/Bigbird/15031-4983-FullBook.docx'
predicted_label = predict_label(file_path, tokenizer, model)
print(predicted_label)


Attention type 'block_sparse' is not possible if sequence_length: 295 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Level 3
